## Loading needed libraries and reading datasets. We later save them in a .py file which is used in other notebooks/scripts as a local library.

In [19]:
#libraries
import pandas as pd
import xarray as xr
import numpy as np

### measurements
data/measurements/fino2.102.01.txt<br />
<b>Format:</b><br />
YYYYmmdd HHMM WSSPD Dir STD<br />
20150101 0000 12.8 100 0.67<br />
20150101 0010 13.2 100 0.89<br />
20150101 0020 14.3 100 0.92<br />
20150101 0030 14.7 100 0.88<br />

In [47]:
def readMeasurements(file):
    data=pd.read_csv(file,sep=" ",header=None)
    data.columns=['date','time','wspd_obs','hgt_obs','std_obs']
    data['time']=data['time'].astype(str).str.zfill(4)
    data['datetime']=pd.to_datetime(data['date'].astype(str)+ ' ' +data['time'].astype(str),format='%Y%m%d %H%M')
    data=data.drop(['date','time'],axis=1)
    data=data.set_index('datetime')

    return data
obs=readMeasurements("data/measurements/fino2.102.01.txt")
obs.describe()

,wspd_obs,hgt_obs,std_obs
count,51417.000000,51417.0,51417.000000
mean,9.780298,100.0,0.673561
std,4.661955,0.0,0.437476
min,0.300000,100.0,0.030000
25%,6.100000,100.0,0.350000
50%,9.300000,100.0,0.560000
75%,13.100000,100.0,0.900000
max,30.500000,100.0,5.420000


## Vortex txt serie, downloaded from interface

<b> Format:</b><br />
Lat=55.00701  Lon=13.15399  Hub-Height=100  Timezone=0   ASL-Height(avg. 3km-grid)=0  (file requested on 2022-03-18 09:32:17)
VORTEX (www.vortex.es) - Computed at 3km resolution based on ERA5 data (designed for correlation purposes)<br />
 
YYYYMMDD HHMM  M(m/s) D(deg)  T(C)  De(k/m3) PRE(hPa)      RiNumber  RH(%)   RMOL(1/m)<br />
20000101 0000    7.7    240    2.1    1.28    1010.0          -1.04   92.6     -0.0163<br />
20000101 0100    7.5    232    1.9    1.28    1009.9          -1.54   93.9     -0.0178<br />
20000101 0200    7.6    224    1.7    1.28    1009.9          -1.71   94.9     -0.0189<br />


In [36]:
def readVortex(file):
    data=pd.read_table(file,skiprows=3,sep=" ",skipinitialspace=True)
    data['HHMM']=['{i:04}'.format(i=i, width=4) for i in data['HHMM']]
    data['datetime']=pd.to_datetime(data['YYYYMMDD'].astype(str)+ ' ' +data['HHMM'].astype(str),format='%Y%m%d %H%M')
    data=data.drop(['YYYYMMDD','HHMM'],axis=1)
    data=data.set_index('datetime')
    data=data.rename(columns={'M(m/s)':'wspd_serie100','D(deg)':'wdir_serie100'})
    return data
vortexSerie100=readVortex("data/vortex/SERIE/vortex.serie.100m.era5.txt")[['wspd_serie100','wdir_serie100']]
vortexSerie100.describe()


,wspd_serie100,wdir_serie100
count,182568.000000,182568.000000
mean,9.742155,202.548628
std,4.425139,89.182528
min,0.000000,0.000000
25%,6.500000,127.000000
50%,9.600000,226.000000
75%,12.700000,272.000000
max,31.300000,360.000000


In [45]:
def readSerie(file):
    netcdf_file=file
    site = xr.open_dataset(netcdf_file)
    site['wspd_serie']=(site['U']**2+site['V']**2)**(1/2)
    site['wdir_serie']=np.arctan(site['V']/site['U'])*360/(2*np.pi)
    return site
file="data/vortex/SERIE/serie.d02.nc"
vortexSerie=readSerie(file)[['wspd_serie','wdir_serie']].interp(lev=100).to_dataframe().reset_index().drop(['lat','lon','lev'],axis=1).set_index('time')
vortexSerie.describe()

,wspd_serie,wdir_serie
count,182568.000000,182568.000000
mean,9.370123,0.812705
std,4.253736,43.413673
min,0.035532,-89.949369
25%,6.248920,-30.468437
50%,9.204796,2.697564
75%,12.218009,31.492699
max,30.052072,89.973954


In [51]:
## Concatenate concurrent period
tmp=obs.merge(vortexSerie100,left_index=True,right_index=True)
merged=tmp.merge(vortexSerie,left_index=True,right_index=True)
merged.describe()

,wspd_obs,hgt_obs,std_obs,wspd_serie100,wdir_serie100,wspd_serie,wdir_serie
count,8568.000000,8568.0,8568.000000,8568.000000,8568.000000,8568.000000,8568.000000
mean,9.790394,100.0,0.674056,10.494596,213.856326,10.093539,0.649191
std,4.674751,0.0,0.439800,4.582545,80.671412,4.404685,41.975633
min,0.300000,100.0,0.030000,0.200000,0.000000,0.214400,-89.903963
25%,6.100000,100.0,0.350000,7.000000,155.000000,6.772138,-27.462077
50%,9.300000,100.0,0.570000,10.300000,236.000000,9.873513,1.299593
75%,13.100000,100.0,0.900000,13.700000,273.000000,13.135153,30.068043
max,29.500000,100.0,4.280000,26.900000,360.000000,25.895206,89.575307
